In [1]:
import numpy as np
import pandas as pd
from skimage.measure import regionprops, label
from scipy.spatial import ConvexHull

In [2]:
from scipy.spatial.distance import directed_hausdorff

import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import nd2
import re

In [3]:
# def parse_filename1(filename):
#     # Определяем шаблон для поиска
#     pattern = r'^(.*?)_LF(\d+)-P(\d+)_(.*?)_(\d+)\.nd2$'

#     # Применяем регулярное выражение к имени файла
#     match = re.match(pattern, filename)

#     if match:
#         # Получаем группы из регулярного выражения
#         group1 = match.group(1)
#         group2 = int(match.group(3))
#         group3 = match.group(4)
#         group4 = int(match.group(5))

#         return group1, group2, group3, group4
#     else:
#         return None

In [4]:
# def parse_filename2(filename):
#     # Определяем шаблон для поиска
#     pattern = r'^(.*?)_(\d+)\.png$'

#     # Применяем регулярное выражение к имени файла
#     match = re.match(pattern, filename)

#     if match:
#         # Получаем группы из регулярного выражения
#         group1 = match.group(1)
#         group2 = int(match.group(2))

#         return group1, group2
#     else:
#         return None

In [5]:
# import shutil
# fn_dir_orig = 'datasets/MSC/30_04_2023-LF1-P6-P21'
# fn_list_orig = [v for v in os.listdir(fn_dir_orig) if v.endswith('.nd2')]
# kv_fn = dict()
# for fn in fn_list_orig:
#     exp, p, marker, n = parse_filename1(fn)

#     kv_fn[(exp, n)] = f'{fn}mask.png'
# fn_dir = 'datasets/MSC/30_04_2023-LF1-P6-P21_masks_upd'
# fn_list = [v for v in os.listdir(fn_dir) if v.endswith('.png')]
# fn_list.sort()
# for fn in fn_list:
#     exp, n = parse_filename2(fn)
#     if (exp, n) in kv_fn.keys():
#         dst_fn = kv_fn[(exp, n)]
#     else:
#         print(fn, 'x')
#     src_fp = os.path.join(fn_dir, fn)
#     dst_fp = os.path.join(fn_dir, dst_fn)
#     print(src_fp, '->', dst_fp)
#     shutil.move(src_fp, dst_fp)

In [6]:
def parse_filename(filename):
    parts = filename.split('_')
    exp = parts[0]
    pacient = parts[1]
    passage = int(parts[2][1:])  # Extracting the number and converting to integer
    number = int(parts[3].split('.')[0])  # Removing the file extension and converting to integer
        
    return exp, pacient, passage, number

In [7]:
def calculate_mse(contour_pts, center, axes, angle):
    # Generate the rotated rectangle that bounds the ellipse
    rect = (center, axes, angle)

    # Generate points on the boundary of the fitted ellipse
    generated_pts = cv2.boxPoints(rect).astype(np.int0)

    # Calculate mean squared error between contour points and ellipse boundary points
    mse = np.mean(np.sum((contour_pts - generated_pts) ** 2, axis=1))
    
    return mse

In [8]:
def calc_ps(passage_mask, contour):
    passage_mask_contour = np.zeros((passage_mask.shape[1], passage_mask.shape[2]))
    cv2.drawContours(passage_mask_contour, [contour], -1, color=1, thickness=cv2.FILLED)
    p1 = passage_mask[0][np.logical_and(passage_mask_contour == 1, passage_mask[0] == 1)].sum()
    p2 = passage_mask[1][np.logical_and(passage_mask_contour == 1, passage_mask[1] == 1)].sum()
    p3 = passage_mask[2][np.logical_and(passage_mask_contour == 1, passage_mask[2] == 1)].sum()
    # total = matrix[np.logical_and(passage_mask_contour == 1, matrix == 1)].sum()
    total = p1 + p2 + p3

    p1 /= total
    p2 /= total
    p3 /= total

    return p1, p2, p3

def get_cell_statistics(matrix, exp, p, pgr, marker, n, passage_mask=None):
    # Convert matrix to uint8 for cv2 operations
    matrix_uint8 = (matrix * 255).astype(np.uint8)

    # Find contours of cells
    contours, _ = cv2.findContours(matrix_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize lists to store statistics
    centers = []
    areas = []
    roundnesses = []
    ellipse_widths = []
    ellipse_heights = []
    angles = []  # New list for storing angles
    hausdorff_distances = []

    if passage_mask is not None:
        p1_list = []
        p2_list = []
        p3_list = []

    # Calculate statistics for each cell
    for contour in contours:
        # Calculate moments to find centroid
        moments = cv2.moments(contour)
        if moments['m00'] != 0:
            center_x = int(moments['m10'] / moments['m00'])
            center_y = int(moments['m01'] / moments['m00'])
    
            # Calculate area
            area = cv2.contourArea(contour)
            if passage_mask is not None:
                p1, p2, p3 = calc_ps(passage_mask, contour)
            
            # Calculate roundness
            perimeter = cv2.arcLength(contour, True)
            roundness = (4 * np.pi * area) / (perimeter ** 2)
    
            # Fit ellipse to the contour
            if contour.shape[0] > 5:
                ellipse = cv2.fitEllipse(contour)

                ellipse_width = ellipse[1][0]
                ellipse_height = ellipse[1][1]

                # Extract the angle of the fitted ellipse
                angle = ellipse[2]

                # Calculate Hausdorff distance between ellipse contour and actual contour
                ellipse_center, ellipse_axes, ellipse_angle = ellipse
                # Get contour points of the fitted ellipse
                ellipse_points = cv2.ellipse2Poly((int(ellipse_center[0]), int(ellipse_center[1])), (int(ellipse_axes[0] / 2), int(ellipse_axes[1] / 2)),
                                                  int(ellipse_angle), 0, 360, 10)
                # Calculate Hausdorff distance between ellipse contour and actual contour
                hausdorff_distance = directed_hausdorff(ellipse_points.reshape(-1, 2), contour.reshape(-1, 2))[0]

                centers.append((center_x, center_y))
                areas.append(area)
                roundnesses.append(roundness)
                ellipse_widths.append(ellipse_width)
                ellipse_heights.append(ellipse_height)
                angles.append(angle)
                hausdorff_distances.append(hausdorff_distance)
                if passage_mask is not None:
                    p1_list.append(p1)
                    p2_list.append(p2)
                    p3_list.append(p3)

    exp_list = [exp] * len(centers)
    p_list = [p] * len(centers)
    pgr_list = [pgr] * len(centers)
    marker_list = [marker] * len(centers)
    n_list = [n] * len(centers)

    res_dict = {
        'Exp': exp_list,
        'P': p_list,
        'PGr': pgr_list,
        'Marker': marker_list,
        'N': n_list,
        'Center': centers,
        'Area': areas,
        'Roundness': roundnesses,
        'Ellipse Width': ellipse_widths,
        'Ellipse Height': ellipse_heights,
        'Angle': angles,
        'Hausdorff Distance': hausdorff_distances,
    }
    if passage_mask is not None:
        res_dict['PGr1_prob'] = p1_list
        res_dict['PGr2_prob'] = p2_list
        res_dict['PGr3_prob'] = p3_list
        pred_p_all = np.stack([p1_list, p2_list, p3_list],axis=0)
        pred_p = np.argmax(pred_p_all, axis=0) + 1
        res_dict['Pred_PGr'] = pred_p.tolist()
        
    # Create pandas DataFrame
    df = pd.DataFrame(res_dict)

    return df

In [9]:
def draw_ellipses(statistics_df, target_size=(1024, 1024), hd_max=10):
    # Create a blank image to draw ellipses on
    contours_image = np.zeros(target_size, dtype=np.uint8)
    ellipses_image = np.zeros(target_size, dtype=np.uint8)
    # Iterate through each row in the DataFrame
    for _, row in statistics_df.iterrows():
        if row['Hausdorff Distance'] <= hd_max:
            # Extract ellipse parameters
            center_x, center_y = map(int, row['Center'])
            if not math.isnan(row['Ellipse Width']):
                ellipse_width = int(row['Ellipse Width'])
                ellipse_height = int(row['Ellipse Height'])
                angle = int(row['Angle'])
        
                # Draw ellipse on the image
                
                cv2.ellipse(ellipses_image, (center_x, center_y), (ellipse_width // 2, ellipse_height // 2), angle, 0, 360, 1, -1)
        
                contours, hierarchy = cv2.findContours(ellipses_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                _ = cv2.drawContours(contours_image, contours, -1, 1, 1)

    return contours_image

In [10]:
# import shutil
# fn_dir = 'datasets/MSC/30_04_2023-LF1-P6-P21_filtered_3classes_test'
# src_dir = 'datasets/MSC/30_04_2023-LF1-P6-P21_filtered_1classes_all'
# dst_dir = 'datasets/MSC/30_04_2023-LF1-P6-P21_filtered_1classes_test'
# fn_list = [v for v in os.listdir(fn_dir) if v.endswith('.npy')]
# for fn in fn_list:
#     src_fp = os.path.join(src_dir, fn)
#     dst_fp = os.path.join(dst_dir, fn)
#     print(src_fp, '->', dst_fp)
#     shutil.copy(src_fp, dst_fp)

In [11]:
# Example usage
# matrix = np.random.randint(0, 2, size=(2, 1024, 1024))

# mask_dir = 'datasets/MSC/30_04_2023-LF1-P6-P21_filtered_1classes_test'
# mask_dir = 'datasets/MSC/30_04_2023-LF1-P6-P21_filtered_1classes_all'
# mask_dir = 'datasets/MSC/30_04_2023-LF1-P6-P21_filtered_3classes_test'
# mask_dir = 'datasets/MSC/30_04_2023-LF1-P6-P21_filtered_3classes_all'
# img_dir = 'datasets/MSC/30_04_2023-LF1-P6-P21_filtered'

# mask_dir = 'datasets/MSC/30_04_2023-LF1-P6-P21_updated_1classes_test'
# mask_dir = 'datasets/MSC/30_04_2023-LF1-P6-P21_updated_1classes_all'
# mask_dir = 'datasets/MSC/30_04_2023-LF1-P6-P21_updated_3classes_test'
# mask_dir = 'datasets/MSC/30_04_2023-LF1-P6-P21_updated_3classes_all'
# img_dir = 'datasets/MSC/30_04_2023-LF1-P6-P21'

mode = 'BG'

marker = mode
mask_dir = os.path.join('results', 'chambers_cropped_orig_size', mode, '3class')
img_dir = 'datasets/second/chambers_cropped'

csv_dir = os.path.join(mask_dir, 'csv')
results_dir = os.path.join(mask_dir, 'results')
imgs_dir = os.path.join(mask_dir, 'imgs_with_masks')

os.makedirs(csv_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)
os.makedirs(imgs_dir, exist_ok=True)

fn_list = [v for v in os.listdir(mask_dir) if v.endswith('.npy')]
fn_list.sort()

color_shift_red = (+100, -100, -100)
color_shift_green = (-200, +200, -200)
color_shift_blue = (-100, -100, +100)
color_shift_white = (+255, +255, +255)

er = 5
dil = er + 2
# dil = 0

for fn in tqdm(fn_list):
    exp, pacient, p, n = parse_filename(fn)

    # img_fp = os.path.join(img_dir, fn[:-4])
    img = plt.imread(os.path.join(img_dir, f'{exp}_{pacient}_p{p}_{"B"}_{n}.png'))[...,2]
    # img = cv2.resize(img, (1024, 1024))
    print(fn, img.shape)
    # img = img / 255
    img = np.stack([img, img, img], axis=2)
    # img = np.zeros((1024, 1024, 3))
    mask_fp = os.path.join(mask_dir, fn)

    with open(mask_fp, 'rb') as f:
        mask = np.load(f)
    mask = cv2.resize(mask.transpose(1, 2, 0), (img.shape[0], img.shape[1])).transpose(2, 0, 1)
    # print(img.shape, mask.shape)
    
    mask_zero = np.zeros_like(mask[0])
    if mask.shape[0] == 2:
        mask = np.stack([mask[1], mask[0], mask_zero, mask_zero], axis=0)
        
        if p in [6, 9, 12, 15, 18, 21]:
            pgr = 1
        else:
            pgr = 0
    else:
        if p in [6, 9]:
            pgr = 1
        elif p in [12, 15]:
            pgr = 2
        elif p in [18, 21]:
            pgr = 3
        else:
            pgr = 0
            
    if mask.shape[0] > 2:
        passage_mask = np.zeros((3, mask.shape[1], mask.shape[2]))
        passage_mask[0] = mask[0]
        passage_mask[1] = mask[2]
        passage_mask[2] = mask[3]
        main_mask = np.zeros((mask.shape[1], mask.shape[2]))
        main_mask[passage_mask[0] == 1] = 1
        main_mask[passage_mask[1] == 1] = 1
        main_mask[passage_mask[2] == 1] = 1
        
    else:
        passage_mask = None
        main_mask = mask[0].copy()
    
    contour_mask = mask[1].copy()
    main_mask[contour_mask == 1] = 0

    if er != 0:
        kernel = np.ones((er, er), np.uint8) 
        main_mask = cv2.erode(main_mask, kernel) 
    if dil != 0:
        kernel = np.ones((dil, dil), np.uint8) 
        main_mask = cv2.dilate(main_mask, kernel) 
    
    statistics_df = get_cell_statistics(main_mask, exp, p, pgr, marker, n, passage_mask=passage_mask)
    statistics_df.to_csv(os.path.join(csv_dir, fn)+'.csv', index=0)
    
    contours_image = draw_ellipses(statistics_df, target_size=(img.shape[0], img.shape[1]), hd_max=50)
    
    result_matrix = img[...,0].copy()
    result_matrix = np.stack([result_matrix, result_matrix, result_matrix], axis=2) * 255
    red_idx = passage_mask[0] == 1
    green_idx = passage_mask[1] == 1
    blue_idx = passage_mask[2] == 1

    white_idx = contours_image == 1
    # blue_idx = contour_mask == 1

    pallete = [[color_shift_red, red_idx],
               [color_shift_green, green_idx],
               [color_shift_blue, blue_idx],
               [color_shift_white, white_idx],
              ]
    for color_shift, color_idx in pallete:
        for c_idx, c in enumerate(color_shift):
            result_matrix[..., c_idx][color_idx] += c

    np.clip(result_matrix, 0, 255, out=result_matrix)
    result_matrix = result_matrix.astype(np.uint8)
        
    plt.imsave(os.path.join(imgs_dir, fn)+'.png', result_matrix, cmap='gray')


  0%|                                                    | 0/20 [00:00<?, ?it/s]

O1_o1_p2_1.npy (921, 921)


  5%|██▏                                         | 1/20 [00:00<00:04,  4.72it/s]

O1_o1_p2_2.npy (945, 945)


 10%|████▍                                       | 2/20 [00:00<00:04,  3.65it/s]

...

 90%|██████████████████████████████████████▋    | 18/20 [00:04<00:00,  3.63it/s]

Y1_y1_p2_8.npy (945, 945)


 95%|████████████████████████████████████████▊  | 19/20 [00:04<00:00,  3.29it/s]

Y1_y1_p2_9.npy (945, 945)


100%|███████████████████████████████████████████| 20/20 [00:04<00:00,  4.01it/s]


In [12]:
csv_list = [v for v in os.listdir(csv_dir) if v.endswith('.csv')]
csv_list.sort()
pd_list = list()
for csv in csv_list:
    pd_list.append(pd.read_csv(os.path.join(csv_dir, csv)))
result_pd = pd.concat(pd_list).reset_index(drop=True)
result_pd['Pred_true'] = result_pd['Pred_PGr'] == result_pd['PGr']
result_pd.to_csv(os.path.join(results_dir, 'result_all.csv'), index=0)
result_pd

,Exp,P,PGr,Marker,N,Center,Area,Roundness,Ellipse Width,Ellipse Height,Angle,Hausdorff Distance,PGr1_prob,PGr2_prob,PGr3_prob,Pred_PGr,Pred_true
0,O1,2,0,BG,1,"(404, 891)",191.0,0.794297,12.154221,22.307146,166.013260,6.082763,0.940828,0.00000,0.059172,1,False
1,O1,2,0,BG,1,"(9, 900)",748.5,0.624475,21.648239,63.807415,176.843048,19.026298,0.992167,0.00000,0.007833,1,False
2,O1,2,0,BG,1,"(172, 885)",2103.0,0.821143,45.066395,60.868454,85.698807,6.708204,0.496239,0.02006,0.483701,1,False
3,O1,2,0,BG,1,"(398, 864)",183.0,0.729597,10.793870,24.065491,166.282150,5.099020,1.000000,0.00000,0.000000,1,False
4,O1,2,0,BG,1,"(75, 834)",1529.5,0.786498,33.851322,62.715965,72.874619,13.000000,0.514371,0.00000,0.485629,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,Y1,2,0,BG,9,"(226, 43)",3450.0,0.477629,58.292259,93.436188,17.780788,14.317821,0.999152,0.00000,0.000848,1,False
949,Y1,2,0,BG,9,"(866, 5)",498.0,0.469469,30.954330,61.896267,68.774574,29.206164,0.479310,0.00000,0.520690,3,False
950,Y1,2,0,BG,9,"(780, 17)",1344.0,0.791422,37.605606,64.323601,20.914726,26.907248,0.796690,0.00000,0.203310,1,False
951,Y1,2,0,BG,9,"(407, 5)",225.0,0.713045,23.011683,24.734705,157.280060,16.278821,1.000000,0.00000,0.000000,1,False


In [13]:
exps = result_pd['Exp'].unique().tolist()
ps = result_pd['P'].unique().tolist()
pgrs = result_pd['PGr'].unique().tolist()
markers = result_pd['Marker'].unique().tolist()
ns = result_pd['N'].unique().tolist()
exps.sort()
ps.sort()
pgrs.sort()
markers.sort()
ns.sort()

In [14]:
for m in markers:
    m_pd = result_pd[result_pd['Marker'] == m].reset_index(drop=True)
    m_pd = m_pd.sort_values(['P'])
    m_pd.to_csv(os.path.join(results_dir, f'result_{m}.csv'), index=0)

In [15]:
ee = []
mm = []
pp = []
ppgr = []
nn = []
counts = []
areas = []
pred_accs = []
# for m in markers:
#     m_pd = result_pd[result_pd['Marker'] == m]
#     for p in ps:
#         p_pd = m_pd[m_pd['P'] == p]
for e in exps:
    e_pd = result_pd[result_pd['Exp'] == e]
    for n in ns:
        n_pd = e_pd[e_pd['N'] == n]
        count = n_pd.reset_index(drop=True).shape[0]
        area = n_pd['Area'].mean()
        pred_acc = n_pd['Pred_true'].mean()
        if count != 0:
            ee.append(n_pd['Exp'].iloc[0])
            pp.append(n_pd['P'].iloc[0])
            ppgr.append(n_pd['PGr'].iloc[0])
            mm.append(n_pd['Marker'].iloc[0])
            nn.append(n_pd['N'].iloc[0])
            counts.append(count)
            areas.append(area)
            pred_accs.append(pred_acc)
count_pd = pd.DataFrame({
        'Exp': ee,
        'P': pp,
        'PGr': ppgr,
        'Marker': mm,
        'N': nn,
        'count': counts,
        'mean area': areas,
        'pred acc': pred_accs,
})
count_pd.to_csv(os.path.join(results_dir, 'result_all_count_area.csv'), index=0)
count_pd

,Exp,P,PGr,Marker,N,count,mean area,pred acc
0,O1,2,0,BG,1,28,1236.839286,0.0
1,O1,2,0,BG,2,49,1544.244898,0.0
2,O1,2,0,BG,3,38,1558.815789,0.0
3,O1,2,0,BG,4,8,1287.812500,0.0
4,O1,2,0,BG,5,2,523.500000,0.0
5,O2,7,0,BG,1,43,1418.186047,0.0
6,O2,7,0,BG,2,8,523.937500,0.0
7,O3,8,0,BG,1,7,2957.000000,0.0
8,O3,8,0,BG,2,9,3119.277778,0.0
9,O3,8,0,BG,3,11,2401.727273,0.0


In [16]:
for m in markers:
    m_pd = count_pd[count_pd['Marker'] == m].reset_index(drop=True)
    m_pd = m_pd.sort_values(['P'])
    m_pd.to_csv(os.path.join(results_dir, f'result_{m}_count_area.csv'), index=0)
    
    print(m)
    display(m_pd)

BG


,Exp,P,PGr,Marker,N,count,mean area,pred acc
0,O1,2,0,BG,1,28,1236.839286,0.0
17,Y1,2,0,BG,8,97,1719.721649,0.0
16,Y1,2,0,BG,7,108,1556.935185,0.0
15,Y1,2,0,BG,6,6,784.000000,0.0
14,Y1,2,0,BG,5,85,1659.905882,0.0
13,Y1,2,0,BG,4,43,1190.755814,0.0
12,Y1,2,0,BG,3,109,1510.091743,0.0
11,Y1,2,0,BG,2,65,1784.276923,0.0
10,Y1,2,0,BG,1,96,1673.125000,0.0
19,Y1,2,0,BG,10,70,1699.035714,0.0


In [17]:
# mask_fp = os.path.join(mask_dir, 'B2_LF1-P6_H2AX_1.nd2mask.png')

# cl = 1
# mask = cv2.imread(mask_fp, cv2.IMREAD_GRAYSCALE)[np.newaxis, ...]
# mask = mask / 255

# mask_contour = mask.copy()
# mask_contour = mask_contour.transpose(1, 2, 0).astype('uint8')
# contours, hierarchy = cv2.findContours(mask_contour,
#                                        cv2.RETR_EXTERNAL,
#                                        cv2.CHAIN_APPROX_SIMPLE)
# _ = cv2.drawContours(mask_contour, contours, -1, 2, 2)

# # mask = mask_contour.transpose(2, 0, 1).astype('float64')
# # assert mask.shape == (1, 1024, 1024)

# mask_contour = mask_contour.transpose(2, 0, 1).astype('float64')
# mask = np.zeros((2, mask.shape[1], mask.shape[2]))
# if cl == 1:
#     cl = 0
# mask[1:2][mask_contour == 2] = 1
# mask[cl:cl + 1][mask_contour == 1] = 1

In [18]:
# statistics_df = get_cell_statistics(mask)
# statistics_df.to_csv('tmp.csv', index=0)
# statistics_df

In [19]:
# # Draw ellipses on the matrix
# contours_image = draw_ellipses(statistics_df)
# result_matrix = mask[0].copy()
# result_matrix[contours_image == 1] = 2

In [20]:
# fig, ax = plt.subplots(figsize=(20, 20))
# ax.imshow(result_matrix)
# plt.show()